[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alucantonio/data_enhanced_simulation/blob/master/2_MDOFsimulator.ipynb)

# Simulation of multiple degrees-of-freedom (MDOF) systems

References:
- Khalid Jawed, "Notes on Discrete Simulation of Slender Structures", chapters 3 and 4.

Recall the equations of motion (with initial conditions) for a MDOF system subject to
conservative, non-conservative, and viscous forces, written as a system of first order ODEs:
$$ \begin{cases}
    \dot{v}_i = \frac{1}{m_i} \left(-\frac{\partial E_p}{\partial q_i} - c v_i
    + F_i\right) \\
    v_i = \dot{q}_i \\
    q_i(0) = q_{i0} \\
    v_i(0) = v_{i0}
\end{cases}
$$
where $q_i$ are the degrees of freedom ($i = 1,..,N$), $v_i$ are the associated
velocities, $m_i$ are the lumped masses, $c$ is the viscous dissipation coefficien,
$F_i$ are the non-conservative forces, and $E_p$ is the potential energy corresponding
to the conservative forces.

In [ ]:
# Use JAX on CPU (needed on MacOS laptop because GPU is not supported)
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"

In [ ]:
import jax.numpy as jnp
from jax import jacfwd, jit, grad, config, vmap
from functools import partial
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import numpy as np

# Set default data type to double precision: important for accurate ODE integration!
config.update("jax_enable_x64", True)

In [ ]:
# Equations of motion (system of first order ODEs to integrate)
@partial(jit, static_argnames=("P",))
def equations_of_motion(t, z, P, m, c=0., F=0.):
    q, v = jnp.array_split(z, 2) # split state into positions and velocity
    dq_dt = v
    dv_dt = -grad(P)(q)/m-c*v/m+F/m
    return jnp.concatenate([dq_dt, dv_dt])

Let us define a function for the Jacobian of the system wrt to the state ($z$), which is
needed for implicit time integration schemes.

In [ ]:
# Jacobian of the rhs of the system of equations of motion
@partial(jit, static_argnames=("P",))
def jacobian(t, z, P, m, c=0., F=0.):
    return jacfwd(equations_of_motion, argnums=1)(t, z, P, m, c, F)

### Example: 2 masses, 1 spring

In [ ]:
# Initial conditions
q0 = jnp.array([0., 1.1])  # initial positions
v0 = jnp.array([0., 0.])  # initial velocities
z0 = jnp.concatenate([q0, v0])

# Masses and stiffnesses
m = jnp.ones(2)

# Potential energy
def S(q):
    return 0.5*(q[1]-q[0] - 1.)**2

# Kinetic energy
def K(v):
    return 0.5*jnp.sum(m * v**2)

# Time span
t_span = (0, 10)  # from t=0 to t=10
t_eval = jnp.linspace(t_span[0], t_span[1], 1000)  # points at which to store the solution

# Solve the system using RK45 (explicit method)
# sol = solve_ivp(equations_of_motion, t_span, z0, args=(S, m), t_eval=t_eval)
# Solve the system using BDF (implicit method)
sol = solve_ivp(equations_of_motion, t_span, z0, args=(S, m), method="BDF", t_eval=t_eval, jac=jacobian)

In [ ]:
# Comparing the performance of explicit/implicit time integrators
print("Explicit time-stepping:")
%timeit sol = solve_ivp(equations_of_motion, t_span, z0, args=(S, m), t_eval=t_eval)
print("Implicit time-stepping:")
%timeit sol = solve_ivp(equations_of_motion, t_span, z0, args=(S, m), method="BDF", t_eval=t_eval, jac=jacobian)

In [ ]:
# Plot the results
plt.plot(sol.t, sol.y[0], label='q0(t)')
plt.plot(sol.t, sol.y[1], label='q1(t)')
plt.plot(sol.t, sol.y[2], label='v0(t)')
plt.plot(sol.t, sol.y[3], label='v1(t)')
plt.xlabel('Time t')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
q, v = np.split(sol.y, 2)
potential_energy = vmap(S, in_axes=(1,))(q)
kinetic_energy = vmap(K, in_axes=(1,))(v)
plt.plot(potential_energy)
plt.plot(kinetic_energy)
plt.plot(potential_energy + kinetic_energy)
plt.show()

### Verlet time integration (optional)

In [ ]:
def velocity_verlet(t, dt, z, P, equations_of_motion, m, c=0., F=0.):
    q, v = jnp.split(z, 2) 
    a = equations_of_motion(t, z, P, m, c, F)[len(q):] 
    q = q + v*dt + 0.5*a*dt**2
    z = jnp.concatenate([q, v])
    new_a = equations_of_motion(t, z, P, m, c, F)[len(q):]
    v = v + 0.5*(new_a + a) * dt
    return jnp.concatenate([q, v])

In [ ]:
z = z0
dt = 0.001
times = np.arange(0., 10., dt)
zs = np.zeros((len(z0), len(times)))
for i,t in enumerate(times):
    z = velocity_verlet(t, dt, z, S, equations_of_motion, m) 
    zs[:,i] = z

In [ ]:
# Plot the results
plt.plot(times, zs[0], label='q0(t)')
plt.plot(times, zs[1], label='q1(t)')
plt.plot(times, zs[2], label='v0(t)')
plt.plot(times, zs[3], label='v1(t)')
plt.xlabel('Time t')
plt.legend()
plt.grid(True)
plt.show()

### Rigid spheres and elastic beam falling in a viscous fluid

In [ ]:
N = 11 # number of spheres (must be odd)
l = 0.1 # total length of the rod
dl = l/(N-1)
xs = jnp.linspace(0., l, N).reshape(-1,1) # array of the x coords as a column
ys = jnp.zeros((N,1))
q0 = jnp.hstack([xs, ys]).ravel() # [x0, y0, x1, y1, ...]
v0 = jnp.zeros(2*N)
z0 = jnp.concatenate([q0, v0])

In [ ]:
# Material parameters
E = 1e9 # Young's modulus
r0 = 1e-3 # beam radius
EA = E*np.pi*r0**2 # stretching stiffenss
EI = E*np.pi*r0**4/4 # bending stiffness
rho_m = 7000. # density of the metal (spheres)
rho_f = 1000. # density of the fluid
radii = 5e-3*np.ones(2*N) # radii of the spheres (2 per dof)
radii[N-1:N+1] = 25e-3
V = 4/3*np.pi*np.power(radii, 3) # volumes of the spheres
m = rho_m * V # masses of the spheres
mu = 1000. # fluid viscosity

# Viscous damping coefficients
c = 6*np.pi*mu*radii

In [ ]:
# Buoyancy forces (TO COMPLETE)
y_components = np.ones(2*N)
# y_components = ... select the dofs corresponding to y components
F = -V*y_components*(rho_m-rho_f)*9.81

Define the functions needed to compute the curvatures and the stretching energy of the system.

In [ ]:
# Geometry functions
def compute_edge_vectors(q):
    # Return the matrix where the rows correspond to the edges and the columns are the x
    # and y components of the edge vectors. Takes as input the array of the node
    # positions (i.e. [x0,y0,x1,y1,...]).
    pass

def compute_curvatures(edge_vectors_norm):
    # Return the array of the discrete curvatures evaluated at the internal nodes of the
    # beam. Takes as an input the matrix of the normalized (unit) edge vectors. 
    # cos_turning_angles = ...
    # sin_turning_angles = ...
    # tan_theta_half = ...
    return 2*tan_theta_half/dl

# Stretching energy
def S(q):
    pass

In [ ]:
# Bending energy
def B(q):
    edge_vectors = compute_edge_vectors(q)
    edge_vectors_norm = edge_vectors / jnp.linalg.norm(edge_vectors, axis=1).reshape(-1,1)
    curvatures = compute_curvatures(edge_vectors_norm)
    return 0.5*EI*jnp.sum(curvatures**2*dl)

# Total potential energy
def P(q):
    pass


Then, we can integrate the equations of motion:

In [ ]:
# Time span
t_span = (0, 10)  # from t=0 to t=10
t_eval = np.linspace(t_span[0], t_span[1], 100)  # points at which to store the solution

# Solve the system using RK45 (explicit method)
# sol = solve_ivp(equations_of_motion, t_span, z0, args=(P, m, c, F), t_eval=t_eval)
# Solve the system using BDF (implicit method)
sol = solve_ivp(equations_of_motion, t_span, z0, args=(P, m, c, F), method="BDF", t_eval=t_eval, jac=jacobian)

In [ ]:
q, v = np.split(sol.y, 2)
plt.plot(sol.t, q[1], label='y1')
plt.plot(sol.t, q[3], label='y2')
plt.plot(sol.t, q[5], label='y3')
plt.xlabel('Time t')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.plot(sol.t, q[0], label='x1')
plt.plot(sol.t, q[2], label='x2')
plt.plot(sol.t, q[4], label='x3')
plt.xlabel('Time t')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot the final configuration of the beam
plt.plot(q[::2,-1],q[1::2,-1])
plt.show()

### Bending of a simply-supported rod under uniform load

In [ ]:
N = 51 # number of nodes
l = 1. # total length of the rod
dl = l/(N-1)
xs = jnp.linspace(0., l, N).reshape(-1,1)
ys = jnp.zeros((N,1))
q0 = jnp.hstack([xs, ys]).ravel()
v0 = jnp.zeros(2*N)
z0 = jnp.concatenate([q0, v0])

In [ ]:
E = 70e9 # Young's modulus
R = 0.013 # outer radius circular cross-section
r = 0.011 # inner radius circular cross-section
I = np.pi/4*(R**4 - r**4)
EA = E*np.pi*(R**2 - r**2) # stretching stiffness
EI = E*I # bending stiffness
rho = 2700 # material's density
m = np.pi*(R**2-r**2)*l*rho/(N-1) # array of nodal massess
c = 5. # damping, just to reach steady-state faster

Modify the equations of motion to enforce the boundary conditions (zero displacement and
velocity on the left end, zero vertical displacement and vertical velocity on the right end).

In [ ]:
@partial(jit, static_argnames=("P",))
def equations_of_motion(t, z, P, m, c=0., F=0.):
    q, v = jnp.array_split(z, 2) # split state into positions and velocity
    # TODO: enforce boundary conditions
    dq_dt = v
    dv_dt = -grad(P)(q)/m-c*v/m+F/m
    return jnp.concatenate([dq_dt, dv_dt])

1. Compute the configuration of the rod in the time interval $[0,0.1]$ when a
   _distributed_ vertical load is applied to it (hint: distribute the load over the nodes). Perform the simulation for each value of the load in the
   interval $[1000,10000]$, with load increment $1000$. (Hint: use the `LSODA` solver in
   `solve_ivp`). 
2. For each load, plot the maximum vertical displacement of the rod as obtained from the
  simulation and from the linear beam theory.
2. Plot the final deformed configuration of the rod (load = 10000).

In [ ]:
@partial(jit, static_argnames=("P",))
def jacobian(t, z, P, m, c=0., F=0.):
    return jacfwd(equations_of_motion, argnums=1)(t, z, P, m, c, F)

In [ ]:
t_span = (0, 0.1)
t_eval = np.linspace(t_span[0], t_span[1], 100)  # points at which to store the solution

y_components = np.ones(2*N)
y_components[::2] = 0. # select the dofs corresponding to y components

y_theory = [] # store the maximum deflection values for each load as computed using the analytical formula
y_num = [] # corresponding numerical values

loads = np.linspace(1000, 10000, 10)

for load in loads:
    print(load)
    # F = ...  compute nodal loads from total load
    # vertical displacement according to linear beam theory
    w = -load/l
    y_theory.append(5*w*l**4/(384*EI))

    # sol = solve_ivp(...)

    q, v = np.split(sol.y, 2)
    # y_num.append(...) # record y-displacement of the midpoint at final time

In [ ]:
# Final configuration (maximum load)
plt.plot(q[::2,-1],q[1::2,-1])
plt.show()

In [ ]:
plt.plot(loads, y_theory, label="linear theory")
plt.plot(loads, y_num, label="numerical model")
plt.xlabel("Total load [N]")
plt.ylabel("Maximum deflection [m]")
plt.legend()
plt.show()

#### Example using Diffrax (optional)

`Diffrax` ([docs](https://docs.kidger.site/diffrax/)) is a JAX-based library providing numerical solvers for differential equations.

In [ ]:
# Needed on Colab (also restart)
!pip install diffrax

In [ ]:
from diffrax import diffeqsolve, ODETerm, ImplicitEuler, PIDController, Heun, Midpoint

def equations_of_motion(t, z, args):
    P, m, c, F = args
    q, v = jnp.array_split(z, 2) # split state into positions and velocity
    q = q.at[0:2].set(0)
    q = q.at[-1].set(0)
    v = v.at[0:2].set(0)
    v = v.at[-1].set(0)
    dq_dt = v
    dv_dt = -grad(P)(q)/m-c*v/m+F/m
    return jnp.concatenate([dq_dt, dv_dt])

term = ODETerm(equations_of_motion)
solver = Heun() # ImplicitEuler()
stepsize_controller = PIDController(rtol=1e-4, atol=1e-4)
P0 = 10000./N # distribute load over nodes
y_components = np.ones(2*N)
y_components[::2] = 0. # select the dofs corresponding to y components
F = -P0*y_components
solution = diffeqsolve(term, solver, t0=0, t1=0.1, dt0=1e-4, y0=z0, args=(P, m, c, F), stepsize_controller=stepsize_controller, max_steps=1000000)

In [ ]:
q, v = np.split(solution.ys[0], 2)
plt.plot(q[::2],q[1::2])
plt.show()